In [3]:
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os
os.environ['GROQ_API_KEY']=os.getenv('GROQ_API_KEY')

def create_dynamic_prompt_template():
    """Creates a dynamic prompt template that can handle various client requirements"""
    base_template = """You are a compassionate outreach coordinator for a recovery center. 
    Generate a personalized SMS message for our client based on the provided information.
    The message should be warm, supportive, and maintain appropriate boundaries.
    No preamble or explanations, just the message.

    Here's an example of the tone and style we're looking for:

    Example SMS:
    Hey {name}, Happy Thanksgiving! We wanted to take a moment to check in on you and see how your recovery journey is going. We're so grateful to have been a part of your path, and we're always here for you if you need support or just someone to talk to.
    Would you be open to speaking at our recovery center about your success? If not, we are always here to help.
    Wishing you a warm and peaceful Thanksgiving, filled with moments of gratitude and connection.
    Take care and know that you're in our thoughts.
    With care,

    Requirements for the message:
    {requirements}

    Important guidelines:
    - Adapt the message for the specified occasion
    - Include any relevant personal details provided
    - Maintain the same warm and supportive tone
    - Keep similar length and structure as the example
    - End with "With care," like the example

    Generate the message now:"""
    return base_template

def format_requirements(requirements: dict) -> str:
    """Formats the requirements dictionary into a readable string"""
    if not requirements:
        return "No specific requirements provided. Generate a general check-in message."
    
    formatted_reqs = []
    # Define key mappings for better prompt formatting
    key_mappings = {
        'name': 'Client Name',
        'occasion': 'Occasion/Event',
        'time_since_recovery': 'Time in Recovery',
        'additional_notes': 'Additional Context',
        'last_contact': 'Last Contact',
        'program_status': 'Program Status',
        'milestone': 'Recent Milestone'
    }
    
    for key, value in requirements.items():
        # Clean the key name and use mapping if available
        clean_key = key.lower().replace('_', ' ')
        display_key = key_mappings.get(clean_key, clean_key.title())
        formatted_reqs.append(f"- {display_key}: {value}")
    
    return "\n".join(formatted_reqs)

def generate_sms(requirements: dict = None) -> str:
    """
    Generates an SMS message based on provided requirements
    
    Args:
        requirements (dict): Dictionary containing message requirements
            Possible keys include:
            - name: Client's name
            - occasion: Special occasion or reason for contact
            - time_since_recovery: Time since starting recovery
            - additional_notes: Any additional context
            - program_status: Current status in the program
            - milestone: Recent achievements or milestones
    """
    # Initialize OpenAI
    llm = ChatGroq(model='llama-3.3-70b-specdec',temperature=0.7)
    
    # Create base prompt template
    base_template = create_dynamic_prompt_template()
    
    # Format requirements
    formatted_reqs = format_requirements(requirements or {})
    
    # Create the final prompt
    prompt = base_template.format(
        name=requirements.get('name', '<name>'),
        requirements=formatted_reqs
    )
    
    # Generate the message
    response = llm.invoke(prompt)
    return response.content

# Example usage
if __name__ == "__main__":
    # Example with different types of requirements
    simple_requirements = {
        "occasion": "New Year",
        "additional_notes": "Has been attending weekly support groups regularly"
    }
    
    detailed_requirements = {
        "name": "Sarah",
        "occasion": "New Year",
        "time_since_recovery": "6 months",
        "program_status": "Completed outpatient program",
        "milestone": "90 days sober",
        "additional_notes": "Recently started mentoring new group members"
    }
    
    # Generate messages with different levels of detail
    print("\nSimple Message:")
    print("-" * 50)
    print(generate_sms(simple_requirements))
    
    print("\nDetailed Message:")
    print("-" * 50)
    print(generate_sms(detailed_requirements))


Simple Message:
--------------------------------------------------
Hey, Happy New Year! We wanted to take a moment to check in on you and see how your recovery journey is going as we start this new chapter. We're so grateful to have you as a part of our weekly support groups, and we're always here for you if you need support or just someone to talk to. Your consistent attendance and commitment to your recovery are truly inspiring, and we're honored to be a part of your path. Would you be open to sharing your story with others, perhaps as a guest speaker at our recovery center? If not, we're always here to help in any way we can. Wishing you a happy, healthy, and fulfilling New Year, filled with hope, growth, and celebration of your accomplishments. Take care and know that you're in our thoughts. With care,

Detailed Message:
--------------------------------------------------
Hey Sarah, Happy New Year! We wanted to take a moment to check in on you and celebrate the incredible progress 